In [1]:
from transformers import AutoTokenizer
from constants import *
import pandas as pd
import evaluate
import os

In [2]:
RESULTS_FOLDER = "results"
REFERENCE_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "reference_code")
ACCEPTED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "accepted_code")
REJECTED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "rejected_code")
HF_TOKEN = open(".token", "r").read().strip()
CODEBLEU = evaluate.load("k4black/codebleu")
BLEU = evaluate.load("bleu")
METEOR = evaluate.load("meteor")
TER = evaluate.load("ter")
ROUGE = evaluate.load('rouge')
CHRF = evaluate.load("chrf")
WER = evaluate.load("wer")

[nltk_data] Downloading package wordnet to /Users/caman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/caman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/caman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
lines = open(os.path.join(RESULTS_FOLDER, "compilation_log.txt"), "r").readlines()

In [4]:
lines[:10]

['2025-01-08 17:12:05 - Compilation and Syntax check failed: deepseekcoder7b-bit_ops-py-c.c\n',
 '2025-01-08 17:12:06 - Compilation successful: codegeex4-str_interp-c-java.java\n',
 '2025-01-08 17:12:06 - Compilation successful: yicoder2b-str_interp-cpp-go.go\n',
 '2025-01-08 17:12:07 - Compilation successful: codeqwen-int_arith-go-cpp.cpp\n',
 '2025-01-08 17:12:07 - Compilation failed: magicoder7b-file_size-java-rs.rs\n',
 '2025-01-08 17:12:07 - Syntax check successful: codegemma7b-file_exists-c-py.py\n',
 '2025-01-08 17:12:07 - Compilation successful: codellama7b-int_factors-rs-c.c\n',
 '2025-01-08 17:12:07 - Compilation successful: deepseekcoderv2-logic_ops-rs-c.c\n',
 '2025-01-08 17:12:08 - Compilation and Syntax check failed: magicoder7b-int_factors-c-cpp.cpp\n',
 '2025-01-08 17:12:08 - Compilation successful: codellama7b-int_cmp-cpp-rs.rs\n']

In [5]:
def check_row_exists(df, m, t, fl, tl):
    return (
        (df["model"] == m)
        & (df["task"] == t)
        & (df["from_lang"] == fl)
        & (df["to_lang"] == tl)
    ).any()

In [6]:
LANG_MAP = {
    "c": "c",
    "cpp": "cpp",
    "go": "go",
    "java": "java",
    "js": "javascript",
    "py": "python",
    "rs": "rust",
}

In [7]:
models = []
tasks = []
from_langs = []
to_langs = []
successes = []
valids = []
codebleus = []
bleus = []
meteors = []
ters = []
rouge1s = []
rouge2s = []
rougeLs = []
rougeLsums = []
chrfs = []
wers = []

if os.path.exists(os.path.join(RESULTS_FOLDER, f"metrics.csv")):
    metrics = pd.read_csv(os.path.join(RESULTS_FOLDER, f"metrics.csv"))
else:
    metrics = pd.DataFrame(
        {
            "model": models,
            "task": tasks,
            "from_lang": from_langs,
            "to_lang": to_langs,
            "success": successes,
            "valid": valids,
            "codebleu": codebleus,
            "bleu": bleus,
            "meteor": meteors,
            "rouge1": rouge1s,
            "rouge2": rouge2s,
            "rougeL": rougeLs,
            "rougeLsum": rougeLsums,
            "chrF": chrfs,
            "ter": ters,
            "wer": wers,
        }
    )

cnt = 0
for line in lines:
    valid = 0
    if "successful" in line:
        valid = 1
    line = line.strip()
    line = [x.strip() for x in line.split(":")][3]
    line = line[: line.rfind(".")]
    line = line.split("-")
    m = line[0]
    t = line[1]
    fl = line[2]
    tl = line[3]
    if check_row_exists(metrics, m, t, fl, tl):
        continue
    cnt += 1
    models.append(m)
    tasks.append(t)
    from_langs.append(fl)
    to_langs.append(tl)
    valids.append(valid)
    # tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_TAGS[m], token=HF_TOKEN, trust_remote_code=True)
    src_code = (
        open(os.path.join(REFERENCE_CODE_FOLDER, f"{t}.{fl}"), "r").read().strip()
    )
    ref_code = (
        open(os.path.join(REFERENCE_CODE_FOLDER, f"{t}.{tl}"), "r").read().strip()
    )
    gen_code = (
        open(os.path.join(ACCEPTED_CODE_FOLDER, f"{m}-{t}-{fl}-{tl}.{tl}"), "r")
        .read()
        .strip()
    )
    print("accepted", m, t, fl, tl, "start ... ", end="")
    bleu = BLEU.compute(predictions=[gen_code], references=[[ref_code]])
    bleus.append(bleu["bleu"])
    meteor = METEOR.compute(predictions=[gen_code], references=[ref_code])
    meteors.append(meteor["meteor"])
    ter = TER.compute(
        predictions=[gen_code], references=[[ref_code]], case_sensitive=True
    )
    ters.append(ter["score"])
    rouge = ROUGE.compute(
        predictions=[gen_code], references=[[ref_code]], tokenizer=lambda x: x.split()
    )
    rouge1s.append(rouge["rouge1"])
    rouge2s.append(rouge["rouge2"])
    rougeLs.append(rouge["rougeL"])
    rougeLsums.append(rouge["rougeLsum"])
    chrf = CHRF.compute(predictions=[gen_code], references=[[ref_code]])
    chrfs.append(chrf["score"])
    wer = WER.compute(predictions=[gen_code], references=[ref_code])
    wers.append(wer)
    codebleu = CODEBLEU.compute(
        predictions=[gen_code],
        references=[[ref_code]],
        lang=[LANG_MAP[tl]],
        weights=(0.25, 0.25, 0.25, 0.25),
    )
    codebleus.append(codebleu["codebleu"])
    print("done")
    successes.append(1)
    if cnt % 10 == 0:
        pd.concat(
            [
                metrics,
                pd.DataFrame(
                    {
                        "model": models,
                        "task": tasks,
                        "from_lang": from_langs,
                        "to_lang": to_langs,
                        "success": successes,
                        "valid": valids,
                        "codebleu": codebleus,
                        "bleu": bleus,
                        "meteor": meteors,
                        "rouge1": rouge1s,
                        "rouge2": rouge2s,
                        "rougeL": rougeLs,
                        "rougeLsum": rougeLsums,
                        "chrF": chrfs,
                        "ter": ters,
                        "wer": wers,
                    }
                ),
            ]
        ).to_csv(os.path.join(RESULTS_FOLDER, f"metrics.csv"), index=False)

for filename in os.listdir(REJECTED_CODE_FOLDER):
    if filename.startswith("."):
        continue
    filename = filename[:filename.rfind(".")]
    filename = filename.split("-")
    m = filename[0]
    t = filename[1]
    fl = filename[2]
    tl = filename[3]
    if check_row_exists(metrics, m, t, fl, tl):
        continue
    models.append(m)
    tasks.append(t)
    from_langs.append(fl)
    to_langs.append(tl)
    successes.append(0)
    valids.append(0)
    codebleus.append(0)
    bleus.append(0)
    meteors.append(0)
    ters.append(0)
    rouge1s.append(0)
    rouge2s.append(0)
    rougeLs.append(0)
    rougeLsums.append(0)
    chrfs.append(0)
    wers.append(0)
    print("rejected", m, t, fl, tl, "done")

metrics = pd.concat(
    [
        metrics,
        pd.DataFrame(
            {
                "model": models,
                "task": tasks,
                "from_lang": from_langs,
                "to_lang": to_langs,
                "success": successes,
                "valid": valids,
                "codebleu": codebleus,
                "bleu": bleus,
                "meteor": meteors,
                "rouge1": rouge1s,
                "rouge2": rouge2s,
                "rougeL": rougeLs,
                "rougeLsum": rougeLsums,
                "chrF": chrfs,
                "ter": ters,
                "wer": wers,
            }
        ),
    ]
)

accepted deepseekcoder7b bit_ops py c start ... done
accepted codegeex4 str_interp c java start ... done
accepted yicoder2b str_interp cpp go start ... done
accepted codeqwen int_arith go cpp start ... done
accepted magicoder7b file_size java rs start ... done
accepted codegemma7b file_exists c py start ... done
accepted codellama7b int_factors rs c start ... done
accepted deepseekcoderv2 logic_ops rs c start ... done
accepted magicoder7b int_factors c cpp start ... 

done
accepted codellama7b int_cmp cpp rs start ... done
accepted granitecode3b file_size cpp js start ... done
accepted codegemma7b dir_make cpp c start ... done
accepted yicoder2b bit_ops js rs start ... 

done
accepted stablecode str_prepend js go start ... done
accepted granitecode3b str_match cpp c start ... done
accepted magicoder7b str_interp py c start ... done
accepted codeqwen logic_ops c cpp start ... done
accepted yicoder9b str_substring py c start ... 

done
accepted yicoder2b int_cmp c rs start ... done
accepted stablecode file_size c rs start ... done
accepted yicoder9b bit_ops py go start ... done
accepted magicoder7b dir_make java go start ... 

done
accepted dolphincoder7b int_factors c java start ... done
accepted yicoder2b int_arith go rs start ... 

...

done
accepted qwen2.5coder2b file_exists java rs start ... done
accepted qwen2.5coder7b dir_make py c start ... done
accepted qwen2.5coder7b int_cmp java js start ... done
accepted qwen2.5coder2b int_arith rs js start ... done
accepted qwen2.5coder7b int_arith js c start ... done
accepted qwen2.5coder2b file_create py go start ... done
accepted qwen2.5coder2b bit_ops js java start ... done
accepted qwen2.5coder7b file_rename rs c start ... done
accepted qwen2.5coder7b str_prepend go cpp start ... done
accepted qwen2.5coder2b str_substring py cpp start ... done
accepted qwen2.5coder7b str_concat cpp js start ... done
accepted qwen2.5coder2b bit_ops cpp rs start ... 

done
accepted qwen2.5coder2b int_cmp cpp java start ... done
accepted qwen2.5coder2b int_factors c rs start ... done
accepted qwen2.5coder7b file_exists go py start ... done
accepted qwen2.5coder2b file_size js cpp start ... done
accepted qwen2.5coder7b str_match js c start ... done
accepted qwen2.5coder7b str_interp js py start ... done
accepted qwen2.5coder2b file_exists java c start ... done
accepted qwen2.5coder7b file_exists rs js start ... done
accepted qwen2.5coder7b file_exists cpp py start ... done
accepted qwen2.5coder2b file_exists java cpp start ... done
accepted qwen2.5coder2b int_arith go py start ... done
accepted qwen2.5coder2b str_interp go c start ... done
accepted qwen2.5coder7b bit_ops java cpp start ... 

done
accepted qwen2.5coder7b str_append py go start ... done
accepted qwen2.5coder7b str_interp py js start ... done
accepted qwen2.5coder7b str_append cpp rs start ... done
accepted qwen2.5coder2b str_interp rs py start ... done
accepted qwen2.5coder2b logic_ops cpp py start ... done
accepted qwen2.5coder7b int_cmp java cpp start ... done
accepted qwen2.5coder7b file_create c java start ... done
accepted qwen2.5coder2b bit_ops go cpp start ... done
accepted qwen2.5coder2b logic_ops py go start ... done
accepted qwen2.5coder2b file_rename js c start ... 

done
accepted qwen2.5coder7b int_cmp java c start ... done
accepted qwen2.5coder2b file_exists c rs start ... done
accepted qwen2.5coder2b file_rename c go start ... done
accepted qwen2.5coder7b str_concat c cpp start ... done
accepted qwen2.5coder2b int_cmp js java start ... 

done
accepted qwen2.5coder7b str_cmp c js start ... done
accepted qwen2.5coder7b file_exists rs cpp start ... done
accepted qwen2.5coder7b str_append js cpp start ... done
accepted qwen2.5coder2b file_create js cpp start ... done
accepted qwen2.5coder2b str_append rs cpp start ... done
accepted qwen2.5coder2b str_append js java start ... done
accepted qwen2.5coder2b str_cmp go cpp start ... done
accepted qwen2.5coder2b logic_ops rs java start ... done
accepted qwen2.5coder2b str_interp go js start ... done
rejected codellama7b str_cmp cpp c done
rejected stablecode str_cmp cpp c done
rejected stablecode bit_ops py c done
rejected codellama7b str_cmp c rs done
rejected codellama7b str_interp go c done
rejected deepseekcoder7b file_rename go js done
rejected granitecode3b file_exists js rs done
rejected codellama7b int_cmp go rs done
rejected granitecode8b str_match js c done
rejected codellama7b int_factors js c done
rejected deepseekcoder7b file_rename cpp js done
rejected granitecode3

In [8]:
metrics.head()

,model,task,from_lang,to_lang,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
0,deepseekcoder7b,bit_ops,py,c,1.0,0.0,0.291374,0.130275,0.362417,0.152866,0.038462,0.101911,0.152866,27.217825,198.000000,2.020202
1,codegeex4,str_interp,c,java,1.0,1.0,0.257701,0.260155,0.687822,0.342857,0.181818,0.342857,0.342857,42.624201,130.769231,1.416667
2,yicoder2b,str_interp,cpp,go,1.0,1.0,0.530565,0.292182,0.648074,0.478873,0.318841,0.478873,0.478873,47.962884,116.000000,1.304348
3,codeqwen,int_arith,go,cpp,1.0,1.0,0.499523,0.239201,0.679556,0.386364,0.206107,0.378788,0.378788,46.138403,214.705882,2.179104
4,magicoder7b,file_size,java,rs,1.0,0.0,0.327270,0.157098,0.257040,0.214286,0.073171,0.214286,0.214286,28.293514,87.931034,0.886792


In [9]:
_ = metrics.info()
metrics.to_csv(os.path.join(RESULTS_FOLDER, f"metrics.csv"), index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10710 entries, 0 to 10709
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   model      10710 non-null  object 
 1   task       10710 non-null  object 
 2   from_lang  10710 non-null  object 
 3   to_lang    10710 non-null  object 
 4   success    10710 non-null  float64
 5   valid      10710 non-null  float64
 6   codebleu   10710 non-null  float64
 7   bleu       10710 non-null  float64
 8   meteor     10710 non-null  float64
 9   rouge1     10710 non-null  float64
 10  rouge2     10710 non-null  float64
 11  rougeL     10710 non-null  float64
 12  rougeLsum  10710 non-null  float64
 13  chrF       10710 non-null  float64
 14  ter        10710 non-null  float64
 15  wer        10710 non-null  float64
dtypes: float64(12), object(4)
memory usage: 1.3+ MB


In [10]:
# per model metrics
col = "model"
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{col}_metrics.csv"), index=False)
m_df

,model,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
5,deepseekcoderv2,0.991597,0.840395,0.364045,0.224655,0.444019,0.310831,0.155442,0.274279,0.308499,39.357484,259.303193,2.879650
14,yicoder9b,1.000000,0.801120,0.348579,0.223868,0.435054,0.311906,0.150798,0.275394,0.309253,38.739161,218.587615,2.462086
0,codegeex4,1.000000,0.798319,0.358695,0.234469,0.441582,0.322318,0.162761,0.286929,0.320034,39.379258,220.404294,2.464054
3,codeqwen,1.000000,0.726891,0.338495,0.209510,0.436202,0.298902,0.134072,0.266471,0.296468,37.286248,207.469820,2.327207
9,magicoder7b,0.998599,0.706872,0.345790,0.225907,0.432927,0.310678,0.151827,0.276961,0.308282,38.302735,206.600143,2.314290
11,qwen2.5coder7b,1.000000,0.689076,0.361523,0.231109,0.438054,0.320139,0.163037,0.285515,0.317612,39.072798,213.667825,2.398706
4,deepseekcoder7b,0.980392,0.671429,0.348562,0.221009,0.429473,0.303430,0.148952,0.269730,0.301163,38.558290,223.427395,2.512550
6,dolphincoder7b,0.990196,0.642150,0.332020,0.212718,0.417652,0.301199,0.143359,0.269154,0.298563,36.822628,208.723302,2.320717
1,codegemma7b,1.000000,0.612045,0.349534,0.243224,0.434448,0.329321,0.165620,0.298748,0.327495,38.574027,162.596389,1.822218
10,qwen2.5coder2b,1.000000,0.598039,0.346990,0.218729,0.428369,0.307419,0.150609,0.273403,0.305137,37.950838,231.645779,2.640206


In [11]:
# per task metrics
col = "task"
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{col}_metrics.csv"), index=False)
m_df

,task,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
10,str_append,0.998413,0.828299,0.367550,0.207255,0.448736,0.364292,0.204215,0.358065,0.361825,34.540294,115.974169,1.271747
12,str_concat,0.998413,0.763116,0.417761,0.286091,0.506756,0.427348,0.273345,0.408102,0.426763,42.284188,124.797244,1.310066
7,int_cmp,0.993651,0.766773,0.376739,0.278366,0.494030,0.451266,0.262989,0.391551,0.449982,42.853422,80.288025,0.848180
13,str_interp,0.995238,0.751196,0.365559,0.237417,0.508984,0.376401,0.209205,0.337580,0.372094,39.473227,136.229353,1.482837
15,str_prepend,1.000000,0.739683,0.342022,0.149086,0.414755,0.314184,0.130488,0.306196,0.313815,27.071814,155.885486,1.715992
6,int_arith,0.998413,0.709062,0.303354,0.251424,0.418450,0.341402,0.153397,0.294005,0.334260,38.830436,126.168415,1.311783
5,file_size,0.996825,0.638535,0.357769,0.223023,0.465541,0.275375,0.122600,0.250817,0.270224,41.456852,267.469361,3.098555
4,file_rename,0.992063,0.608000,0.382596,0.296926,0.520136,0.224027,0.104231,0.214620,0.223335,49.130375,347.998577,3.843041
3,file_exists,0.998413,0.602544,0.369400,0.202545,0.429099,0.207590,0.099554,0.181865,0.206480,38.161306,631.532970,8.869049
14,str_match,0.993651,0.594249,0.271125,0.112156,0.284695,0.194424,0.051099,0.144550,0.191730,29.187124,197.520461,2.283288


In [12]:
# per from_lang metrics
col = "from_lang"
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{col}_metrics.csv"), index=False)
m_df

,from_lang,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
1,cpp,0.995425,0.688772,0.353619,0.231824,0.437437,0.312984,0.157158,0.280529,0.310893,38.523301,226.973971,2.563953
6,rs,0.996732,0.674754,0.320415,0.204162,0.409207,0.295984,0.141403,0.264419,0.294518,37.004784,211.342466,2.399628
0,c,0.991503,0.677653,0.344712,0.209716,0.424644,0.296453,0.144014,0.264576,0.294544,37.393356,257.587806,2.916532
2,go,0.992810,0.662936,0.348791,0.220832,0.438417,0.311810,0.151141,0.278584,0.308938,38.418673,185.039313,2.108793
3,java,0.995425,0.591596,0.335430,0.208624,0.411001,0.305871,0.140966,0.272422,0.302953,35.910100,186.307828,2.051721
5,py,0.990196,0.555116,0.351372,0.222646,0.431132,0.317228,0.149015,0.281488,0.314557,37.821895,210.989185,2.315745
4,js,0.996078,0.543307,0.347294,0.233216,0.436963,0.319256,0.161771,0.285882,0.316835,38.705471,230.494437,2.574980


In [13]:
# per to_lang metrics
col = "to_lang"
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{col}_metrics.csv"), index=False)
m_df

,to_lang,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
4,js,0.994118,0.917817,0.315276,0.156369,0.377295,0.228282,0.087130,0.201557,0.226842,29.048200,221.705276,2.624011
5,py,0.996732,0.893770,0.285097,0.142118,0.368637,0.229187,0.085859,0.204716,0.226975,32.473539,235.251307,2.904442
3,java,0.997386,0.735256,0.375911,0.244929,0.460684,0.325316,0.177653,0.299396,0.324301,43.885764,298.429298,3.560847
2,go,0.996732,0.516066,0.352861,0.218433,0.417341,0.333543,0.169185,0.299238,0.332699,39.115765,299.151117,2.988461
0,c,0.990196,0.480528,0.316587,0.254252,0.443395,0.322256,0.163547,0.280748,0.319879,39.853480,145.489588,1.541524
1,cpp,0.992810,0.448321,0.377441,0.282633,0.488653,0.401034,0.225880,0.359509,0.395374,42.794436,158.224244,1.697924
6,rs,0.990196,0.400000,0.378468,0.232681,0.432974,0.320261,0.136359,0.282936,0.317447,36.605222,149.269381,1.599443


In [14]:
# per model and task specific metrics
col = ["model", "task"]
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{"_".join(col)}_metrics.csv"), index=False)
m_df

,model,task,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
92,deepseekcoderv2,int_cmp,1.00000,1.000000,0.387340,0.276039,0.514029,0.449316,0.256752,0.384421,0.447392,43.760173,88.181662,0.937706
100,deepseekcoderv2,str_prepend,1.00000,1.000000,0.380369,0.154246,0.460438,0.308639,0.138386,0.295809,0.308639,28.694890,191.916534,2.158659
248,yicoder9b,str_append,1.00000,1.000000,0.407101,0.248547,0.474624,0.406289,0.247218,0.399316,0.403310,37.600978,103.157334,1.136932
95,deepseekcoderv2,str_append,1.00000,0.976190,0.400921,0.238782,0.481148,0.388831,0.233260,0.379162,0.383922,37.030591,124.538516,1.406335
91,deepseekcoderv2,int_arith,1.00000,0.976190,0.307229,0.252206,0.436197,0.357080,0.162504,0.303980,0.351121,41.177207,124.620068,1.310287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,stablecode,file_exists,1.00000,0.047619,0.370352,0.181242,0.401812,0.194493,0.094395,0.168766,0.193552,36.367371,740.279077,10.597922
204,stablecode,bit_ops,0.97619,0.048780,0.303330,0.225271,0.384865,0.270239,0.116365,0.224164,0.265450,31.810983,144.984246,1.477037
218,stablecode,str_match,1.00000,0.047619,0.245714,0.103582,0.265567,0.178524,0.044528,0.135386,0.176534,27.840106,161.421559,1.851936
212,stablecode,int_factors,1.00000,0.023810,0.258412,0.149513,0.312972,0.300082,0.100890,0.239409,0.298392,32.902936,136.197330,1.419097


In [15]:
# per from_lang and to_lang specific metrics
col = ["from_lang", "to_lang"]
m1_df = (
    metrics[metrics["success"] > 0]
    .groupby(col)[[x for x in metrics.columns if x not in ["model", "task", "from_lang", "to_lang", "success"]]]
    .mean()
    .reset_index()
)
m2_df = (
    metrics.groupby(col)
    .success.mean()
    .reset_index()
)
m_df = m2_df.merge(m1_df, how="left", on=col)
m_df["rank"] = m_df["valid"] * m_df["success"]
m_df = m_df.sort_values(["rank"], ascending=False)
m_df = m_df.drop("rank", axis=1)
m_df.to_csv(os.path.join(RESULTS_FOLDER, f"{"_".join(col)}_metrics.csv"), index=False)
m_df

,from_lang,to_lang,success,valid,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
9,cpp,js,0.996078,0.933071,0.322150,0.178690,0.415171,0.241039,0.098400,0.218462,0.238715,31.643981,207.453864,2.488164
21,java,js,0.996078,0.925197,0.325213,0.160529,0.361155,0.257218,0.107241,0.230028,0.255643,30.301689,205.234842,2.454139
40,rs,js,0.996078,0.913386,0.285288,0.147191,0.360016,0.204700,0.072017,0.179748,0.203596,28.370277,226.731412,2.731668
15,go,js,0.992157,0.916996,0.317742,0.159537,0.393192,0.233853,0.089425,0.207504,0.232565,29.594112,195.297489,2.212431
34,py,js,0.988235,0.916667,0.326397,0.153817,0.382141,0.230868,0.082149,0.199888,0.230201,27.362072,230.290003,2.707202
22,java,py,1.000000,0.905882,0.296964,0.138062,0.367184,0.232539,0.080706,0.215194,0.229004,31.604503,216.953806,2.622838
10,cpp,py,1.000000,0.901961,0.281815,0.151757,0.378925,0.234879,0.086132,0.201708,0.233061,33.073253,237.959367,2.932310
41,rs,py,0.992157,0.905138,0.259997,0.117110,0.324624,0.210332,0.069971,0.188330,0.209439,29.750457,231.165266,2.878986
3,c,js,0.996078,0.901575,0.314960,0.138445,0.352194,0.202052,0.073516,0.173721,0.200384,27.005943,265.187677,3.149494
28,js,py,0.996078,0.893701,0.311072,0.162066,0.397610,0.240650,0.101101,0.217342,0.239188,33.561076,245.282885,3.078051


In [16]:
metrics[metrics["success"] > 0][
    ["codebleu", "bleu", "meteor", "rouge1", "rouge2", "rougeL", "rougeLsum", "chrF", "ter", "wer"]
].describe()

,codebleu,bleu,meteor,rouge1,rouge2,rougeL,rougeLsum,chrF,ter,wer
count,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000,10646.000000
mean,0.343077,0.218717,0.426963,0.308510,0.149355,0.275413,0.306175,37.682262,215.525361,2.418689
std,0.136706,0.161233,0.183012,0.172338,0.148436,0.169986,0.172268,15.182734,399.671243,4.604608
min,0.013580,0.000000,0.024424,0.000000,0.000000,0.000000,0.000000,2.091041,0.000000,0.000000
25%,0.246590,0.104948,0.292271,0.181818,0.041841,0.150943,0.179220,26.871229,82.352941,0.857143
50%,0.338087,0.187455,0.400872,0.285714,0.106010,0.239376,0.283019,36.299142,100.000000,1.062500
75%,0.428048,0.301560,0.536523,0.413178,0.216216,0.373071,0.410256,46.453637,200.000000,2.134544
max,1.000000,1.000000,0.999998,1.000000,1.000000,1.000000,1.000000,100.000000,10100.000000,98.000000
